## Data & Preprocessing

In [1]:
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from tqdm import tqdm
import re, random
import pandas as pd

punkt_param = PunktParameters()
abbreviation = ['тыс', 'руб', 'т.е', 'ул', 'д', 'сек', 'мин', 'т.к', 'т.н', 'т.о', 'ср', 'обл', 'кв', 'пл',
                'напр', 'гл', 'и.о', 'им', 'зам', 'гл', 'т.ч']
punkt_param.abbrev_types = set(abbreviation)  # Для правильного разбиения на предложения
sent_tokenizer = PunktSentenceTokenizer(punkt_param).tokenize
word_tokenizer = RegexpTokenizer(r'\w+')
stop_words = get_stop_words('russian')

In [2]:
df = pd.DataFrame(columns=['topic', 'essay'])

In [3]:
path = '/Users/maks/PycharmProjects/Projects/mad/topic_modeling/data/full_essays.txt'
with open(path, 'r') as file:
    text = file.read()

In [4]:
blocks = re.split('<s>Тема: ', text)[1:-1]

In [5]:
for ind, values in enumerate(blocks):
    topic, essay = values.split('\nСочинение: ')
    df.loc[ind] = [topic, essay.replace('</s>\n', '')]

## TextRank

In [6]:
from itertools import combinations
from nltk.stem.snowball import RussianStemmer
import networkx as nx
import math


def similarity(s1, s2):
    """
    :param s1: Множество слов предложения s1
    :param s2: Множество слов предложения s2
    :return float: Степень схожести предложений
    """
    if not len(s1) or not len(s2):
        return 0.0
    return len(s1.intersection(s2))/(math.log(len(s1)+1) + math.log(len(s2)+1))


def text_rank(text):
    """
    :param text: Исходный текст
    :return list: Список троек (номер предложения i, ранг этого предложения pr[i], предложение s),
             отсортированный по убыванию ранга
    """
    sentences = sent_tokenizer(text)  # Список предложений текста
    if len(sentences) < 2:
        s = sentences[0]
        return [(1, 0, s)]
    words = [set(RussianStemmer().stem(word) for word in word_tokenizer.tokenize(sentence.lower())
                 if word not in stop_words) for sentence in sentences]
    # Список множеств слов с примененным стеммингом, без стоп-слов и приведенных к нижнему регистру.
    # Элемент списка - множество слов одного предложения

    # Нумерация предложений с 0 до len(sent) и генерация всех возможных комбинаций по два:
    pairs = combinations(range(len(sentences)), 2)
    scores = [(i, j, similarity(words[i], words[j])) for i, j in pairs]
    scores = filter(lambda x: x[2], scores)  # Фильтр совсем непохожих предложений

    g = nx.Graph()
    g.add_weighted_edges_from(scores)  # Создание графа с весами ребер
    pr = nx.pagerank(g)  # Словарь: ключ - номер вершины, значение - её ранг

    return sorted(((i, pr[i], s) for i, s in enumerate(sentences) if i in pr),
                  key=lambda x: pr[x[0]], reverse=True)  # Сортировка по убыванию ранга тройки


def text_rank_extract(text, n=5):
    tr = text_rank(text)
    top_n = sorted(tr[:n])  # Сортировка первых n предложений по их порядку в тексте
    return ' '.join(x[2] for x in top_n)  # Соединяем предложения

## LSA

In [7]:
from nltk.stem.snowball import RussianStemmer
import numpy
from numpy.linalg import svd
import math


def create_dictionary(text):
    """
    :param text: Исходный текст
    :return dict: Словарь, в котором ключ - слово, значение - номер слова для нумерации строки матрицы
    """
    words = set(RussianStemmer().stem(word) for word in word_tokenizer.tokenize(text.lower()) if word not in stop_words)
    return dict((w, i) for i, w in enumerate(words))


def create_matrix(text, dictionary):
    """
    Создает матрицу размерности |уникальные слова|х|предложения|,
    где элемент (ij) - количество появлений слова i в предложении j.
    :param text: Исходный текст
    :param dictionary: Словарь (слово, номер слова)
    :return matrix: Матрица размерности |уникальные слова|х|предложения|,
                    где элемент (ij) = 0, если слова i нет в предложении j,
                                     = частота слова i в тексте.
    """
    sentences = sent_tokenizer(text)

    words_count = len(dictionary)
    sentences_count = len(sentences)

    matrix = numpy.zeros((words_count, sentences_count))
    for col, sentence in enumerate(sentences):
        for word in word_tokenizer.tokenize(sentence.lower()):
            word = RussianStemmer().stem(word)
            if word in dictionary:  # Стоп-слова не учитываются в матрице
                row = dictionary[word]
                matrix[row, col] += 1
    rows, cols = matrix.shape
    if rows and cols:
        word_count = numpy.sum(matrix)  # Количество слов в тексте
        for row in range(rows):
            unique_word_count = numpy.sum(matrix[row, :])  # Количество определенного слова в тексте
            for col in range(cols):
                if matrix[row, col]:
                    matrix[row, col] = unique_word_count/word_count
    else:
        matrix = numpy.zeros((1, 1))
    return matrix


def compute_ranks(matrix, n):
    """
    :param matrix: Матрица терм-предложение с TF в элементах.
    :param n: Количество предложений в реферате.
    :return list: Список рангов предложений.
    """
    u_m, sigma, v_m = svd(matrix, full_matrices=False)
    powered_sigma = tuple(s**2 if i < n else 0.0 for i, s in enumerate(sigma))

    ranks = []
    # Итерации по столбцам матрицы (т.е. строкам траспонированной)
    for column_vector in v_m.T:
        rank = sum(s*v**2 for s, v in zip(powered_sigma, column_vector))  # По формуле из статьи lsa2
        ranks.append(math.sqrt(rank))
    return ranks


def lsa_extract(text, n=5):
    """
    Создает реферат методом LSA.
    :param text: Исходный текст
    :param n: Количество предложений в реферате
    :return string: Реферат
    """
    d = create_dictionary(text)
    m = create_matrix(text, d)
    r = compute_ranks(m, n)
    sentences = sent_tokenizer(text)
    rank_sort = sorted(((i, r[i], s) for i, s in enumerate(sentences)), key=lambda x: r[x[0]], reverse=True)
    top_n = sorted(rank_sort[:n])
    return ' '.join(x[2] for x in top_n)

## k-means

In [8]:
# Алгоритм k-means для суммаризации
from nltk.stem.snowball import RussianStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer


class StemTokenizer(object):
    def __init__(self):
         self.st = RussianStemmer()

    def __call__(self, doc):
        return [self.st.stem(t) for t in word_tokenizer.tokenize(doc.lower())]


def get_distances_to_clusters(text, n):

    sentences = sent_tokenizer(text)  # Разбиение на предложения
    # Генерация tfidf с учетом стемминга и стоп-слов, термы - униграммы.
    tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words=stop_words, tokenizer=StemTokenizer()).fit_transform(sentences)
    km = KMeans(n_clusters=n, random_state=0).fit_transform(tfidf)  # Выполнение k-means
    return km  # Матрица n x k, где n - количество предложений, k - количество кластеров


def get_list_sent(text, n):  # Возвращает список предложений, наиболее близких к каждому из кластеров.
    cluster_matrix = get_distances_to_clusters(text, n)
    id_list = []
    for i in range(n):
        id_list.append(cluster_matrix[:, i].argmin())
    return sorted(id_list)


def kmeans_extract(text, n=5):
    sentences = sent_tokenizer(text)
    count = len(sentences)
    if n <= count:
        l = get_list_sent(text, n)
        return ' '.join(s for i, s in enumerate(sentences) if i in l)
    else:
        return text  # Если заданное количество предложений в будущем реферате больше, чем количество предложений в тексте, то возвращаем сам текст

In [9]:
# применим алгоритм textRank, LSA, k-means для всех сочинений
df['textRank'], df['LSA'], df['k-means'] = None, None, None
num_sent = 30
for ind in tqdm(range(len(df))):
    df['textRank'].iloc[ind] = text_rank_extract(df.iloc[ind]['essay'], num_sent)
    df['LSA'].iloc[ind] = lsa_extract(df.iloc[ind]['essay'], num_sent)    
    df['k-means'].iloc[ind] = lsa_extract(df.iloc[ind]['essay'], num_sent)    

100%|██████████| 399/399 [00:31<00:00, 12.56it/s]


## Let's see the results

In [10]:
ind = random.randint(0, len(df))
print(f'Topic:\n {df.iloc[ind]["topic"]}\n')
print(f'Essay:\n {df.iloc[ind]["essay"]}\n')
print(f'By textRank:\n {df.iloc[ind]["textRank"]}\n')
print(f'By LSA:\n {df.iloc[ind]["LSA"]}\n')
print(f'By k-means:\n {df.iloc[ind]["k-means"]}\n')

Topic:
 «Прогресс – это движение по кругу, но все более быстрое». Л.Левинсон.

Essay:
 Человечество находится в постоянном движении. Развивается наука, техника, человеческий разум, и если сравнить первобытность и наши дни, то видно, что человеческое общество прогрессирует. От первобытного стада мы пришли к государству, от примитивных орудий труда – к совершенной технике, и если раньше человек не мог объяснить такие природные явления, как гроза или смена года, то к настоящему моменту он уже освоил космос. Исходя из этих соображений, я не могу согласиться с точкой зрения Л.Левинсона на прогресс как на циклическое движение. На мой взгляд, такое понимание истории означает топтание на месте без продвижения вперед, постоянное повторение. Время никогда не повернётся вспять, какие бы факторы ни способствовали регрессу. Человек всегда решит любую проблему и не допустит вымирания своего рода. Конечно, в истории всегда были подъёмы и спады, и поэтому я считаю, что график человеческого прогресса –

In [12]:
with open('/Users/maks/Documents/Хрестоматии 1, 2/researches/исходники/1.txt', 'r') as file:
    text = file.read()
    text = text_rank_extract(text, 5)
text

'Законы суть условия, на которых люди, существовавшие до того независимо и изолированно друг от друга, объединились в общество. Совокупность всех частей свободы, пожертвованных на общее благо, составила верховную власть народа, а суверен стал законным ее хранителем и распорядителем. Потребовалось воздействовать на чувства, чтобы воспрепятствовать эгоистическим поползновениям души каждого отдельного индивида ввергнуть законы общества в пучину первобытного хаоса. Это воздействие на чувства служит наказанием нарушителям законов. Я говорю “воздействовать на чувства”, ибо, как показал опыт, массы не в состоянии ни усвоить твердые правила поведения, ни противостоять всеобщему закону разложения, проявление которого наблюдается и в мире физических явлений, и в сфере морали.'